In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib3
from swiftshadow.classes import Proxy

In [2]:
# Importing dataframe
df = pd.read_excel('data/to_scrap.xlsx')

In [9]:
df.head(5)

,Identifiant,MainNodeId,Version,Création,Modification,Société cible ou acteur,Siret,Email de la cible,Site internet de la cible,Région,...,Enterprise Value (USD),Equity Value (USD),Revenue (USD),EBITDA (USD),EBIT (USD),Earnings (USD),Revenue Multiple,EBITDA Multiple,EBIT Multiple,Earnings Multiple
0,495207,515967,54,2024-08-08 00:00:00,2024-11-16 00:00:00,AETHER FINANCIAL SERVICES,81147500000000,abernard@aetherfs.com,https://aetherfs.com/,Île-de-France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,498547,519330,34,2024-09-19 00:00:00,2024-11-16 00:00:00,ODERIS,48971900000000,stephanie.yakhou@oderis.fr,http://www.oderis.fr/,Île-de-France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,474997,495552,40,2024-02-29 00:00:00,2024-10-26 00:00:00,BEX CAPITAL,83825700000000,contact@bexcapital.com,http://www.bexcapital.com/,Région Sud - Provence-Alpes-Côte d'Azur,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,465515,486018,83,2023-12-14 00:00:00,2024-11-09 00:00:00,FINANCIÈRE ARBEVEL (VOIR MONTPENSIER ARBEVEL),41468300000000,flhenoret@arbevel.com,https://www.arbevel.com/fr,Île-de-France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,453216,473647,131,2023-09-05 00:00:00,2024-11-02 00:00:00,NEXTSTAGE AM,44266700000000,info-investor@nextstage.com,https://nextstage-am.com/,Île-de-France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Disabling warnings about ssl certificated because we will be making requests without ssl (cause some website have expired ssl certificates we need to ignore)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
zebi = Proxy( autoRotate= True)

2024-12-03 21:40:00,604 - swiftshadow [INFO]:Loaded proxies from cache


In [5]:
print(zebi.proxy())

2024-12-03 21:41:04,491 - swiftshadow [INFO]:Cache expired. Updating cache.
['165.155.228.10:9480', 'http']


In [ ]:
proxy = zebi.proxy()




url = "http://httpbin.org/ip"  # This site returns your public IP for testing
response = requests.get(url, proxies={proxy[1]:proxy[0] )

print(response.json())

TypeError: expected string or bytes-like object, got 'list'

In [14]:
def scrape_company_text(url):
    try:
        # Fetch webpage content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }  # Simulate being a browser to not be blocked by website
        response = requests.get(url, headers=headers, timeout=10, verify=False)

        # Detect encoding if not provided by the server
        response.encoding = (
            "utf-8"  # Make sure all characters such as é à etc are correctly rendered
        )

        response.raise_for_status()  # Raise error for bad responses

        # Parse the HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract title and meta description
        title = soup.title.string if soup.title else ""
        meta_desc = ""
        if soup.find("meta", attrs={"name": "description"}):
            meta_desc = soup.find("meta", attrs={"name": "description"}).get("content", "")

        # Extract text from headers and paragraphs
        headers = [h.get_text(strip=True) for h in soup.find_all(["h1", "h2", "h3"])]
        paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]

        # Combine all extracted text
        extracted_text = f"Title: {title} \\n Meta Description: {meta_desc} \\n "
        extracted_text += "\\n".join(headers + paragraphs)
        cleaned_text = " ".join(extracted_text.split())
        return cleaned_text

    except Exception as e:
        print(f"Error: {e}")
        return "Error"

In [6]:
# Example usage
url = "http://www.oderis.fr"
company_text = scrape_company_text(url)
print(company_text)

Title: Oderis - Conseil en création de valeur Meta Description: Cabinet indépendant de conseil financier en transaction et accompagnement opérationnel, pour les fonds d'investissements et dirigeants. Pays/région Pays/région Cabinet indépendanten transaction et accompagnement opérationnel 1erconseil LBOFrance 1erconseil Small CapFrance 2èmeconseil Mid CapFrance 10èmeconseil LBOEurope Oderis portedes valeurs fortes et différenciantessur son marché Entrepreneurship RSE & ESG Business partner Human Focused Nosmétiers Transaction Services Restructuring Transformation Évaluation Tax & Legal ESG 150 Consultants à vos côtés, etheureux de l’être! Oderis enquelques chiffres Collaborateurs Bureaux Associés et directeurs Lignes de service Nos clientsparlent de nous Nosactualités Oderis poursuit sa mue à l’international Oderis poursuit sa mue à l’international Oderis renforce encore son bureau lyonnais Oderis renforce encore son bureau lyonnais Oderis lance son offre ESG avec Radia Benhallam Oderis

In [12]:
df["Site internet de la cible"]

0                      https://aetherfs.com/
1                      http://www.oderis.fr/
2                 http://www.bexcapital.com/
3                 https://www.arbevel.com/fr
4                  https://nextstage-am.com/
                        ...                 
1567       http://www.lucie-saint-clair.com/
1568    http://www.alkos-group.com/index.php
1569          http://www.france-thermes.com/
1570              http://www.alliagecare.fr/
1571                https://www.porsolt.com/
Name: Site internet de la cible, Length: 1572, dtype: object

In [15]:
df["Homepage Text"] = df["Site internet de la cible"].apply(
    lambda x: scrape_company_text(x) if x is not np.nan else x
)

Error: Invalid URL '|': No scheme supplied. Perhaps you meant https://|?
Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error: HTTPSConnectionPool(host='www.slat.com', port=443): Read timed out. (read timeout=10)
Error: 404 Client Error: Not Found for url: https://www.riskeco.com/
Error: 500 Server Error: Internal Server Error for url: https://www.hgh.fr/
Error: HTTPConnectionPool(host='www.intermas-el.fr', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002AC4F6D04A0>: Failed to resolve 'www.intermas-el.fr' ([Errno 11001] getaddrinfo failed)"))
Error: 404 Client Error: Not Found for url: https://groupehbf.com/fr/
Error: HTTPConnectionPool(host='www.thomson-networks.com', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002AC4CE4FFE0>, 'Connection to www.thomson-networks.com timed out

In [16]:
df["Homepage Text"]

0       Title: Aether Financial Services - Leveraging ...
1       Title: Oderis - Conseil en création de valeur ...
2       Title: None \n Meta Description: \n The leadin...
3       Title: ARBEVEL – Une maison de recherche au se...
4       Title: Accueil - NextStage AM, Spécialiste du ...
                              ...                        
1567                                                Error
1568    Title: Who we are | KDC/ONE \n Meta Descriptio...
1569    Title: Accueil - France Thermes \n Meta Descri...
1570    Title: Alliage Care, Management d'établissemen...
1571    Title: Home - PORSOLT \n Meta Description: Por...
Name: Homepage Text, Length: 1572, dtype: object

In [17]:
df.to_excel("data/resultats.xlsx")

In [21]:
df.loc[df["Homepage Text"] == 'Error','Homepage Text']

63      Error
72      Error
85      Error
87      Error
94      Error
        ...  
1531    Error
1546    Error
1549    Error
1553    Error
1567    Error
Name: Homepage Text, Length: 153, dtype: object

In [22]:
df.to_csv('data/resultats.csv')